In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import geopandas as gpd

## World population by continents


In [ ]:
world = pd.read_csv('data/world_population.csv')
world_cp = world.copy()
world_cp.head()

In [ ]:
# drop unnecessary columns
world_cp = world_cp.drop(['Rank', 'CCA3', 'Country/Territory', 'Capital', 'Area (km²)', 'Density (per km²)', 'Growth Rate'], axis=1)

# lowercase all the columns
world_cp.columns = world_cp.columns.str.lower()

# replace the spaces with underscore
world_cp.columns = world_cp.columns.str.replace(' ', '_')

# rename the columns
world_cp = world_cp.rename(columns={'2022_population': '2022', '2020_population': '2020', '2015_population': '2015', '2010_population': '2010', '2000_population': '2000', '1990_population': '1990', '1980_population': '1980', '1970_population': '1970'})

# reorder the columns
world_cp = world_cp[['continent', '1970','1980', '1990', '2000', '2010', '2015', '2020', '2022']]
world_cp.head()

### Line Graph


In [ ]:
plt.figure(figsize=(10, 6))

for continent in world_cp['continent'].unique():
    df_continent = world_cp[world_cp['continent'] == continent]
    plt.plot(df_continent.columns[1:], df_continent.values[0][1:], marker='o', label=continent)

plt.xlabel('Year')
plt.ylabel('Population')
plt.title('Population Growth by Continent')
plt.xticks(rotation=45)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
raw_df = pd.read_csv('./data/FAOSTAT_data.csv')
raw_df.head()

Food insecurity:

* Prevalence of severe food insecurity in the total population (percent): 210401 (M/F)
* Prevalence of moderate or severe food insecurity in the total population (percent): 210091 (M/F)
* Number of severely food insecure people (million): 210071 (M/F)
* Number of moderately or severely food insecure people (million): 210081 (M/F)

## Further cleaning and preprocessing


In [ ]:
raw_df.columns = raw_df.columns.str.lower()
raw_df.columns = raw_df.columns.str.replace(' ', '_')
# change the area column to country column
raw_df = raw_df.rename(columns={'area': 'country'})

In [ ]:
# filter out the data by item code
# prevalence of severe food insecurity in the total population
prev_severe_total = raw_df[raw_df['item_code'] == '210401'] 
prev_severe_male = raw_df[raw_df['item_code'] == '210401M']
prev_sever_female = raw_df[raw_df['item_code'] == '210401F']

# prevalence of moderate or severe food insecurity in the total population
prev_mod_total = raw_df[raw_df['item_code'] == '210091']
prev_mod_male = raw_df[raw_df['item_code'] == '210091M']
prev_mod_female = raw_df[raw_df['item_code'] == '210091F']

In [ ]:
prev_severe_total.head()

In [ ]:
# read the data
code_df = pd.read_csv('./data/country_annotation.csv')

# drop unnecessary columns
code_df = code_df.drop(columns=['code', 'languages'])

# rename the columns
code_df = code_df.rename(columns={'name':'country'})

code_df.head()

In [ ]:
# copy the dataframe
df = prev_severe_total.copy()
# change the value column type to numeric value
df['value'] = pd.to_numeric(df['value'], errors='coerce')

In [ ]:
# group by the country and get the sum of all the values
df = df.groupby('country', as_index=False)['value'].sum().reset_index(drop=True)
df.head()

In [ ]:
# merge the region data with the main data
df = pd.merge(df, code_df, on='country', how='left')

In [ ]:
# drop the rows with value 0
df = df[df['value'] != 0].reset_index(drop=True)
df

In [ ]:
# dictionary mapping each country to its continent
country_to_continent = {
    'Antigua and Barbuda': 'North America',
    'Bahamas': 'North America',
    'Barbados': 'North America',
    'Belize': 'North America',
    'Cabo Verde': 'Africa',
    'Congo': 'Africa',
    'Czechia': 'Europe',
    'Côte d\'Ivoire': 'Africa',
    'Eswatini': 'Africa',
    'Grenada': 'North America',
    'Haiti': 'North America',
    'Iran (Islamic Republic of)': 'Asia',
    'Jamaica': 'North America',
    'Lao People\'s Democratic Republic': 'Asia',
    'Myanmar': 'Asia',
    'Netherlands (Kingdom of the)': 'Europe',
    'Republic of Korea': 'Asia',
    'Republic of Moldova': 'Europe',
    'Russian Federation': 'Europe',
    'Saint Kitts and Nevis': 'North America',
    'Saint Lucia': 'North America',
    'Saint Vincent and the Grenadines': 'North America',
    'Sao Tome and Principe': 'Africa',
    'Trinidad and Tobago': 'North America',
    'United Kingdom of Great Britain and Northern Ireland': 'Europe',
    'United Republic of Tanzania': 'Africa',
    'United States of America': 'North America',
    'Viet Nam': 'Asia'
}

# map the country to the continent
df['continent'] = df['continent'].fillna(df['country'].map(country_to_continent))

df

In [ ]:
# data sanity check
# check the continent value
df.continent.unique()

# replace the incorrect value
df['continent'] = df['continent'].str.replace('North Americ', 'North America')
df['continent'] = df['continent'].str.replace('North Americaa', 'North America')

In [ ]:
df

## Data Visualiztion: Prevalence of severe food insecurity in the total population

### Nested pie chart


In [ ]:
# plot a nested pie chart using plotly
fig = px.sunburst(df, path=['continent', 'country'], values='value', names = 'country', title='Prevalence of severe food insecurity in the total population',color_discrete_sequence=px.colors.qualitative.Set3)
# change the plot size
fig.update_layout(width=800, height=800)
fig.show()

### Packed Bubble chart


In [ ]:
# group the df by continent and aggregate the data
grouped_df = df.groupby('continent').apply(lambda x: x[['country', 'value']].to_dict(orient='records')).reset_index(name='data')

# convert the grouped DataFrame to a dictionary
result_dict = grouped_df.set_index('continent').to_dict(orient='index')

print(result_dict)

---


In [ ]:
# load the data
cleaned_df = pd.read_csv('./data/CleanedData/foodinsecuritybygender.csv', encoding='ISO-8859-1')
# convert it to dataframe
cleaned_df = pd.DataFrame(cleaned_df)
cleaned_df.head()

In [ ]:
# fill the null values with 0
cleaned_df['AVG_PercentFemale']= cleaned_df['AVG_PercentFemale'].fillna(0)
cleaned_df['AVG_PercentMale']= cleaned_df['AVG_PercentMale'].fillna(0)

# convert the data type to numeric
cleaned_df['AVG_PercentFemale'] = pd.to_numeric(cleaned_df['AVG_PercentFemale'], errors='coerce')
cleaned_df['AVG_PercentMale'] = pd.to_numeric(cleaned_df['AVG_PercentMale'], errors='coerce')

# add a new boolean column if the primarycolonizer column is not null
cleaned_df['colonizer'] = cleaned_df['PrimaryColonizer'].notnull()

cleaned_df

In [ ]:
# drop the year column
another_df = cleaned_df.drop(columns=['YearPeriod', 'Region', 'Unnamed: 0', 'PrimaryColonizer', 'colonizer'])
# group by the country and get the sum of all the values
another_df = another_df.groupby('Area', as_index=False).sum().reset_index(drop=True)
another_df.head()

In [ ]:
# save the dataframe to a csv file
another_df.to_csv('./data/avg_info.csv', index=False)

Colors:

Europe: #140D4F
Africa: #598B2C

#11270B
#42CAFD
#FFA400
#F5EDF0
#1C0B19
#66B3BA
#92140C


```{ojs}
chart = BubbleChart(files, {
  label: d => [...d.id.split(".").pop().split(/(?=[A-Z][a-z])/g), d.value.toLocaleString("en")].join("\n"),
  value: d => d.value,
  group: d => d.id.split(".")[1],
  title: d => `${d.id}\n${d.value.toLocaleString("en")}`,
  link: d => `https://github.com/prefuse/Flare/blob/master/flare/src/${d.id.replace(/\./g, "/")}.as`,
  width: 1152
})
```

```{ojs}
flare = FileAttachment("./data/avg_info.csv").csv({typed: true})
```

```{ojs}
files = flare.filter(d => d.value !== null)
```

```{ojs}
// Copyright 2021-2023 Observable, Inc.
// Released under the ISC license.
// https://observablehq.com/@d3/bubble-chart
function BubbleChart(data, {
  name = ([x]) => x, // alias for label
  label = name, // given d in data, returns text to display on the bubble
  value = ([, y]) => y, // given d in data, returns a quantitative size
  group, // given d in data, returns a categorical value for color
  title, // given d in data, returns text to show on hover
  link, // given a node d, its link (if any)
  linkTarget = "_blank", // the target attribute for links, if any
  width = 640, // outer width, in pixels
  height = width, // outer height, in pixels
  padding = 3, // padding between circles
  margin = 1, // default margins
  marginTop = margin, // top margin, in pixels
  marginRight = margin, // right margin, in pixels
  marginBottom = margin, // bottom margin, in pixels
  marginLeft = margin, // left margin, in pixels
  groups, // array of group names (the domain of the color scale)
  colors = d3.schemeTableau10, // an array of colors (for groups)
  fill = "#ccc", // a static fill color, if no group channel is specified
  fillOpacity = 0.7, // the fill opacity of the bubbles
  stroke, // a static stroke around the bubbles
  strokeWidth, // the stroke width around the bubbles, if any
  strokeOpacity, // the stroke opacity around the bubbles, if any
} = {}) {
  // Compute the values.
  const D = d3.map(data, d => d);
  const V = d3.map(data, value);
  const G = group == null ? null : d3.map(data, group);
  const I = d3.range(V.length).filter(i => V[i] > 0);

  // Unique the groups.
  if (G && groups === undefined) groups = I.map(i => G[i]);
  groups = G && new d3.InternSet(groups);

  // Construct scales.
  const color = G && d3.scaleOrdinal(groups, colors);

  // Compute labels and titles.
  const L = label == null ? null : d3.map(data, label);
  const T = title === undefined ? L : title == null ? null : d3.map(data, title);

  // Compute layout: create a 1-deep hierarchy, and pack it.
  const root = d3.pack()
      .size([width - marginLeft - marginRight, height - marginTop - marginBottom])
      .padding(padding)
    (d3.hierarchy({children: I})
      .sum(i => V[i]));

  const svg = d3.create("svg")
      .attr("width", width)
      .attr("height", height)
      .attr("viewBox", [-marginLeft, -marginTop, width, height])
      .attr("style", "max-width: 100%; height: auto; height: intrinsic;")
      .attr("fill", "currentColor")
      .attr("font-size", 10)
      .attr("font-family", "sans-serif")
      .attr("text-anchor", "middle");

  const leaf = svg.selectAll("a")
    .data(root.leaves())
    .join("a")
      .attr("xlink:href", link == null ? null : (d, i) => link(D[d.data], i, data))
      .attr("target", link == null ? null : linkTarget)
      .attr("transform", d => `translate(${d.x},${d.y})`);

  leaf.append("circle")
      .attr("stroke", stroke)
      .attr("stroke-width", strokeWidth)
      .attr("stroke-opacity", strokeOpacity)
      .attr("fill", G ? d => color(G[d.data]) : fill == null ? "none" : fill)
      .attr("fill-opacity", fillOpacity)
      .attr("r", d => d.r);

  if (T) leaf.append("title")
      .text(d => T[d.data]);

  if (L) {
    // A unique identifier for clip paths (to avoid conflicts).
    const uid = `O-${Math.random().toString(16).slice(2)}`;

    leaf.append("clipPath")
        .attr("id", d => `${uid}-clip-${d.data}`)
      .append("circle")
        .attr("r", d => d.r);

    leaf.append("text")
        .attr("clip-path", d => `url(${new URL(`#${uid}-clip-${d.data}`, location)})`)
      .selectAll("tspan")
      .data(d => `${L[d.data]}`.split(/\n/g))
      .join("tspan")
        .attr("x", 0)
        .attr("y", (d, i, D) => `${i - D.length / 2 + 0.85}em`)
        .attr("fill-opacity", (d, i, D) => i === D.length - 1 ? 0.7 : null)
        .text(d => d);
  }

  return Object.assign(svg.node(), {scales: {color}});
}
```

```{ojs}
import {BubbleChart} from "@d3/bubble-chart-component"
import {howto} from "@d3/example-components"
import {Swatches} from "@d3/color-legend"
```

In [ ]:
# #| vscode: {languageId: python}
# # browser_market_share = {
# #     'browsers': ['firefox', 'chrome', 'safari', 'edge', 'ie', 'opera'],
# #     'market_share': [8.61, 69.55, 8.36, 4.12, 2.76, 2.43],
# #     'color': ['#5A69AF', '#579E65', '#F9C784', '#FC944A', '#F24C00', '#00B825']
# # }

# colors = {'color': ['#11270B', '#42CAFD', '#FFA400', '#F5EDF0', '#1C0B19', '#66B3BA', '#92140C']}


# class BubbleChart:
#     def __init__(self, area, bubble_spacing=0):
#         """
#         Setup for bubble collapse.

#         Parameters
#         ----------
#         area : array-like
#             Area of the bubbles.
#         bubble_spacing : float, default: 0
#             Minimal spacing between bubbles after collapsing.

#         Notes
#         -----
#         If "area" is sorted, the results might look weird.
#         """
#         area = np.asarray(area)
#         r = np.sqrt(area / np.pi)

#         self.bubble_spacing = bubble_spacing
#         self.bubbles = np.ones((len(area), 4))
#         self.bubbles[:, 2] = r
#         self.bubbles[:, 3] = area
#         self.maxstep = 2 * self.bubbles[:, 2].max() + self.bubble_spacing
#         self.step_dist = self.maxstep / 2

#         # calculate initial grid layout for bubbles
#         length = np.ceil(np.sqrt(len(self.bubbles)))
#         grid = np.arange(length) * self.maxstep
#         gx, gy = np.meshgrid(grid, grid)
#         self.bubbles[:, 0] = gx.flatten()[:len(self.bubbles)]
#         self.bubbles[:, 1] = gy.flatten()[:len(self.bubbles)]

#         self.com = self.center_of_mass()

#     def center_of_mass(self):
#         return np.average(
#             self.bubbles[:, :2], axis=0, weights=self.bubbles[:, 3]
#         )

#     def center_distance(self, bubble, bubbles):
#         return np.hypot(bubble[0] - bubbles[:, 0],
#                         bubble[1] - bubbles[:, 1])

#     def outline_distance(self, bubble, bubbles):
#         center_distance = self.center_distance(bubble, bubbles)
#         return center_distance - bubble[2] - \
#             bubbles[:, 2] - self.bubble_spacing

#     def check_collisions(self, bubble, bubbles):
#         distance = self.outline_distance(bubble, bubbles)
#         return len(distance[distance < 0])

#     def collides_with(self, bubble, bubbles):
#         distance = self.outline_distance(bubble, bubbles)
#         return np.argmin(distance, keepdims=True)

#     def collapse(self, n_iterations=50):
#         """
#         Move bubbles to the center of mass.

#         Parameters
#         ----------
#         n_iterations : int, default: 50
#             Number of moves to perform.
#         """
#         for _i in range(n_iterations):
#             moves = 0
#             for i in range(len(self.bubbles)):
#                 rest_bub = np.delete(self.bubbles, i, 0)
#                 # try to move directly towards the center of mass
#                 # direction vector from bubble to the center of mass
#                 dir_vec = self.com - self.bubbles[i, :2]

#                 # shorten direction vector to have length of 1
#                 dir_vec = dir_vec / np.sqrt(dir_vec.dot(dir_vec))

#                 # calculate new bubble position
#                 new_point = self.bubbles[i, :2] + dir_vec * self.step_dist
#                 new_bubble = np.append(new_point, self.bubbles[i, 2:4])

#                 # check whether new bubble collides with other bubbles
#                 if not self.check_collisions(new_bubble, rest_bub):
#                     self.bubbles[i, :] = new_bubble
#                     self.com = self.center_of_mass()
#                     moves += 1
#                 else:
#                     # try to move around a bubble that you collide with
#                     # find colliding bubble
#                     for colliding in self.collides_with(new_bubble, rest_bub):
#                         # calculate direction vector
#                         dir_vec = rest_bub[colliding, :2] - self.bubbles[i, :2]
#                         dir_vec = dir_vec / np.sqrt(dir_vec.dot(dir_vec))
#                         # calculate orthogonal vector
#                         orth = np.array([dir_vec[1], -dir_vec[0]])
#                         # test which direction to go
#                         new_point1 = (self.bubbles[i, :2] + orth *
#                                       self.step_dist)
#                         new_point2 = (self.bubbles[i, :2] - orth *
#                                       self.step_dist)
#                         dist1 = self.center_distance(
#                             self.com, np.array([new_point1]))
#                         dist2 = self.center_distance(
#                             self.com, np.array([new_point2]))
#                         new_point = new_point1 if dist1 < dist2 else new_point2
#                         new_bubble = np.append(new_point, self.bubbles[i, 2:4])
#                         if not self.check_collisions(new_bubble, rest_bub):
#                             self.bubbles[i, :] = new_bubble
#                             self.com = self.center_of_mass()

#             if moves / len(self.bubbles) < 0.1:
#                 self.step_dist = self.step_dist / 2

#     def plot(self, ax, labels, colors):
#         """
#         Draw the bubble plot.

#         Parameters
#         ----------
#         ax : matplotlib.axes.Axes
#         labels : list
#             Labels of the bubbles.
#         colors : list
#             Colors of the bubbles.
#         """
#         for i in range(len(self.bubbles)):
#             circ = plt.Circle(
#                 self.bubbles[i, :2], self.bubbles[i, 2], color=colors[i])
#             ax.add_patch(circ)
#             ax.text(*self.bubbles[i, :2], labels[i],
#                     horizontalalignment='center', verticalalignment='center')


# bubble_chart = BubbleChart(area=data_dict['AVG_PercentFemale'],
#                            bubble_spacing=0.1)

# bubble_chart.collapse()

# fig, ax = plt.subplots(subplot_kw=dict(aspect="equal"))
# bubble_chart.plot(ax, data_dict['Area'], colors)
# ax.axis("off")
# ax.relim()
# ax.autoscale_view()
# ax.set_title('Food Insecurity')

# plt.show()